In [8]:
import random
import itertools

import pandas as pd
import numpy as np

from operator import itemgetter

# Genetic Search Algorithm

In [9]:
def point_mutation(lineup, rate, person_lookup, formation, week):
    '''
    Description:
        Randomly change a players inside of a lineup

    Input:
        lineup(list): list of players
        rate(dbl): the rate at which the mutation happens
        person_lookup(dict): Players broken down by week and position
        formation(tuple): formation for the players
        week(int): The week for of the season

    Output:
        child(list): The line up with the mutation changes that occured (If any)
    '''

    # Get the number of players by position that need to be on the field
    defense, midfield, forward = formation

    # Flags to check if the randomly selected mutation matches the player that is already in the lineup.
    gkp_match, def_match, mid_match, fwd_match = (True, True, True, True)
    
    child = lineup.copy()

    # Loop through all of the players in the lineup
    for idx, j in enumerate(lineup):
        # Check to see if the random number is less than the mutation rate.
        # If the number is less that the rate, procees to change the lineup at the partiucular index.
        if random.uniform(0, 1) < rate:
            # Goal Keeper
            if idx == 0:
                while gkp_match:
                    gkp_idx = int(random.sample(range((len(person_lookup[week]['GKP'])-1)), 1)[0])

                    if person_lookup[week]['GKP'][gkp_idx] == child[idx]:
                        continue
                    else:
                        gkp_match = False
                        child[idx] = person_lookup[week]['GKP'][gkp_idx]
            # Defender
            elif idx <= defense:
                while def_match:
                    def_idx = int(random.sample(range((len(person_lookup[week]['DEF'])-1)), 1)[0])

                    if person_lookup[week]['DEF'][def_idx] == child[idx]:
                        continue
                    else:
                        def_match = False
                        child[idx] = person_lookup[week]['DEF'][def_idx]
            # Midfield
            elif idx <= (defense+midfield):
                while mid_match:
                    mid_idx = int(random.sample(range((len(person_lookup[week]['MID'])-1)), 1)[0])

                    if person_lookup[week]['MID'][mid_idx] == child[idx]:
                        continue
                    else:
                        mid_match = False
                        child[idx] = person_lookup[week]['MID'][mid_idx]

            else:
                # Forward
                while fwd_match:
                    fwd_idx = int(random.sample(range((len(person_lookup[week]['FWD'])-1)), 1)[0])

                    if person_lookup[week]['FWD'][fwd_idx] == child[idx]:
                        continue
                    else:
                        fwd_match = False
                        child[idx] = person_lookup[week]['FWD'][fwd_idx]


    return child

def crossover(parent1, parent2, rate, formation):
    '''
    Description:
        mutate a new child based off of the combination of two parents.

    Input:
        parent1(list): list of lineups
        parent2(list): list of lineups
        rate(dbl): the rate at which the muatation happens. If a random number
        is greater than the rate, no mutation will occur.
        formation(tuple): the formation that the team is playing.

    Output:
        a possible combination of the two parents.
    '''

    # The number of players by position
    defense, midfield, forward = formation

    # Flags to determine if the players repeat within the two parents
    def_same_flag, def_same_count = (True, 0)
    mid_same_flag, mid_same_count = (True, 0)
    fwd_same_flag, fwd_same_count = (True, 0)

    # If the mutation rate is lower than the overall rate,
    # take part of parent 1 and part of parent two and combine them 
    # into a new child

    # Goalie
    if random.uniform(0, 1) >= rate:
        return parent1['lineup']
    else:
        # 50-50 chance to choose the goalie from parent 1 or parent 2
        if random.randint(0,1) == 0:
            new_gkp = [parent1['lineup'][0]]
        else:
            new_gkp = [parent2['lineup'][0]]

        # Defense
        # Choose a random number that descides how many players from parent 1 or parent 2
        # should be taken
        parent1_def = random.randint(0, defense)
        parent2_def = defense - parent1_def 

        # If no players from parent 1 should be taken, then use all of parent 2
        # If no players from parent 2 should be taken, then use all of parent 1
        if parent1_def == 0:
            new_def = parent2['lineup'][1:(defense+1)]
        elif parent2_def == 0:
            new_def = parent1['lineup'][1:(defense+1)]
        else:
            # Try and sample the players from the two parents. If the players are the 
            # same between the two parents, then resample. Repeat this process 5 times.
            # If any of the players are still the same, randomly select parent 1 or parent 2. (Logic below)
            while (def_same_flag == True) and (def_same_count < 4):
                def_random_parent1 = random.sample(range(0, (defense-1)), parent1_def)
                def_random_parent2 = random.sample(range(0, (defense-1)), parent2_def)

                def_sample_parent_1 = list(np.array(parent1['lineup'][1:(defense+1)])[def_random_parent1])
                def_sample_parent_2 = list(np.array(parent2['lineup'][1:(defense+1)])[def_random_parent2])

                def_same_check = sum([i in def_sample_parent_2 for i in def_sample_parent_1])

                if def_same_check > 0:
                    def_same_count += 1
                else:
                    def_same_flag = False
                    new_def = def_sample_parent_1 + def_sample_parent_2
        
        # Midfield
        # Same logic as above for midfield
        parent1_mid = random.randint(0, midfield)
        parent2_mid = midfield - parent1_mid 

        if parent1_mid == 0:
            new_mid = parent2['lineup'][(defense+1):(defense+midfield+1)]
        elif parent2_mid == 0:
            new_mid = parent1['lineup'][(defense+1):(defense+midfield+1)]
        else:
            while (mid_same_flag == True) and (mid_same_count < 4):
                mid_random_parent1 = random.sample(range(0, (midfield-1)), parent1_mid)
                mid_random_parent2 = random.sample(range(0, (midfield-1)), parent2_mid)

                mid_sample_parent_1 = list(np.array(parent1['lineup'][(defense+1):(defense+midfield+1)])[mid_random_parent1])
                mid_sample_parent_2 = list(np.array(parent2['lineup'][(defense+1):(defense+midfield+1)])[mid_random_parent2])

                mid_same_check = sum([i in mid_sample_parent_2 for i in mid_sample_parent_1])

                if mid_same_check > 0:
                    mid_same_count += 1
                else:
                    mid_same_flag = False
                    new_mid = mid_sample_parent_1 + mid_sample_parent_2

        # Forward
        # If there is only one forward on the field, there is a 50-50 chance of
        # selecting them from parent 1 or parent 2.

        # If there is more than one forward; Same logic as above for forwards
        if forward == 1:
            if random.randint(0, 1) == 0:
                new_fwd = list(parent1['lineup'][-1:])
            else:
                new_fwd = list(parent2['lineup'][-1:])
        else:
            parent1_fwd = random.randint(0, forward)
            parent2_fwd = forward - parent1_fwd 

            if parent1_fwd == 0:
                new_fwd = parent2['lineup'][(defense+midfield+1):]
            elif parent2_fwd == 0:
                new_fwd = parent1['lineup'][(defense+midfield+1):]
            else:
                while (fwd_same_flag == True) and (fwd_same_count < 5):
                    fwd_random_parent1 = random.sample(range(0, (forward-1)), parent1_fwd)
                    fwd_random_parent2 = random.sample(range(0, (forward-1)), parent2_fwd)

                    fwd_sample_parent_1 = list(np.array(parent1['lineup'][(defense+midfield+1):])[fwd_random_parent1])
                    fwd_sample_parent_2 = list(np.array(parent2['lineup'][(defense+midfield+1):])[fwd_random_parent2])

                    fwd_same_check = sum([i in fwd_sample_parent_2 for i in fwd_sample_parent_1])

                    if fwd_same_check > 0:
                        fwd_same_count += 1
                    else:
                        fwd_same_flag = False
                        new_fwd = fwd_sample_parent_1 + fwd_sample_parent_2
        
        # If the random sampling from the teams did not result in a new unique team
        # (IE there is repats in the positions), than randomly select the position
        # from parent 1 or parent 2.
        if fwd_same_count >= 4:
            if random.randint(0, 1) == 0:
                new_fwd = parent1['lineup'][(defense+midfield+1):]
            else:
                new_fwd = parent2['lineup'][(defense+midfield+1):]

        if mid_same_count >= 4 :
            if random.randint(0, 1) == 0:
                new_mid = parent1['lineup'][(defense+1):(defense+midfield+1)]
            else:
                new_mid = parent2['lineup'][(defense+1):(defense+midfield+1)]

        if def_same_count >= 4:
            if random.randint(0, 1) == 0:
                new_def = parent1['lineup'][1:(defense+1)]
            else:
                new_def = parent2['lineup'][1:(defense+1)]

        # Return the concatenated list of the new players.
        return new_gkp + new_def + new_mid + new_fwd

def reproduce(selected, pop_size, p_cross, p_mutation, formation, person_lookup, week):
    '''
    Description:
        Take parent strings a combine them combine/mutate them into children strings.

    Input:
        selected(list): list of dictionaries that are potential best lineups
        pop_size(int): number of candiadtes to consider
        p_cross(dbl): rate at which the parent to parent-to-parent crossover happens
        p_mutation(dbl): rate at which the single point mutation will happen (changing random players)
        person_lookup(dict): Players broken down by week and position
        formation(tuple): formation for the players
        week(int): The week for of the season 

    Output:
        children(list): list of all the children created from the parents.
    '''
    children = []
    # Selects the items before (Odd indicies) and after (even indicies) 
    # for mutation later
    for i, p1 in enumerate(selected):
        if i % 2 == 0:
            p2 = selected[i+1]
        else:
            p2 = selected[i-1]

        if i == (len(selected)-1):
            p2 = selected[0]

        child = {}
        # Tries to combine the two parents into new children
        child['lineup'] = crossover(p1, p2, p_cross, formation)

        # Randomly mutates points within the child
        child['lineup'] = point_mutation(child['lineup'], p_mutation, person_lookup, formation, week)
        children.append(child)
    
        if len(children) == pop_size:
            break

    return children

def binary_tournament(pop):
    '''
    Description:
        The function will randomly select two different lists from the population pool.
        It will then compared the cost of the two list against each other. Which ever
        list has a better cost, they get selected and move on in the selection process.

    Input:
        pop(list): List of dictionaries containing the cost and lineups for each item under
        consideration

    Output:
        dictionary containg the list that won the "tournament"

        {bitstring: [],
         fitness: int}
    '''
    # Randomly select two bitstrings from the overall list
    i, j = random.randint(0, (len(pop)-1)), random.randint(0, (len(pop)-1))

    # Check to see if the indicies are the same
    while i == j:
        j =  random.randint(0, len(pop)-1)

    # Select the string with the highest cost to move forward.
    if pop[i]['fitness'] >= pop[j]['fitness']:
        return pop[i]
    else: 
        return pop[j]

def onemax(lineup, player_scores_weeks, week):
    '''
    The cost of the algorithm is the sum of the player costs 
    '''
    scores = [player_scores_weeks[week][i] for i in lineup]
    return sum(scores)

def random_lineup(formation, person_lookup, week):
    '''
    Description:
        Create a random list of players

    Input:
        formation(tuple): tuple containing the number of defenders, midfielders and forwards in the formation
        person_lookup(dict): Dictionary containing the names of the players for each position

    Output:
        A list of randomly generated players.
    '''
    goalie = 1
    defense, midfield, forward = formation

    goalie_idx = random.sample(range(0, (len(person_lookup[week]['GKP'])-1)), 1)
    defense_idx = random.sample(range(0, (len(person_lookup[week]['DEF'])-1)), defense)
    midfield_idx = random.sample(range(0, (len(person_lookup[week]['MID'])-1)), midfield)
    forward_idx = random.sample(range(0, (len(person_lookup[week]['FWD'])-1)), forward)

    goalies = list(person_lookup[week]['GKP'][goalie_idx])
    defenders = list(person_lookup[week]['DEF'][defense_idx])
    midies = list(person_lookup[week]['MID'][midfield_idx])
    forwards = list(person_lookup[week]['FWD'][forward_idx])

    return goalies + defenders + midies + forwards

def search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week):
    '''
Description:
    This function contians the main logic of the program. It will search through
    the various combinations of children and parents to find a soultion that will
    optimize the given cost functions. In this case, the optimal solution is a 
    list of all ones.

Input:
    max_gens(int): The number of generations to loop through to find an optimal
    solution.
    num_bits(int): The number of bits contained in the list
    pop_size(int): The number of children to be children to be considered against the parent.
    p_crossover(dbl): THe probability of keeping the parent vs mixing aprent and child.
        ex) p_crossover = .99; probability of keeping parent to next generation = .01
    p_mutation(dbl): The probability of changing a single position in a list.

Output:
    best(list): A list of a possible optimal solution.  
    '''
    early_stop = 0

    # Create a random set of possible solutions
    population = [{'lineup':random_lineup(formation, person_lookup, week)} for i in range(pop_size)]
    
    # Find the "cost" of each colution
    for idx,_ in enumerate(population):
        population[idx]['fitness'] = onemax(population[idx]['lineup'], player_scores_weeks, week)

    # Sort the lists to find the solution with the highest cost, and keep the largest as the best solution
    best = sorted(population, key=itemgetter('fitness'), reverse=True)[0]

    # # Loop through the number of generations
    for gen in range(max_gens):
        old_fitness = best['fitness']

        # Randomly select possible offspring
        selected = [binary_tournament(population) for _ in range(len(population))]
        # Reproduce children based off of the parent 
        children = reproduce(selected, pop_size, p_crossover, p_mutation, formation, person_lookup, week)

        for idx, _ in enumerate(children):
            children[idx]['fitness'] = onemax(children[idx]['lineup'], player_scores_weeks, week)

        sort_children =  sorted(children, key=itemgetter('fitness'), reverse=True) 

        # If the best child has a higher cost than current, replace the current with child
        if sort_children[0]['fitness'] >= best['fitness']:
            best = sort_children[0].copy()

        population = sort_children.copy()

        print('Gen: {0}, Fitness: {1}, Best: {2} '.format(gen, best['fitness'], best['lineup']))

        if old_fitness == best['fitness']:
            early_stop += 1
        else:
            early_stop = 0

        if early_stop == 5:
            break

    return best

# Data Prep

In [10]:
def normalize_data(col):
    '''
    Helper function to normalize a array of data betwene 0 (minimum) and 1 (maximum)

    Input:
        col(list/series): columns from a dataframe that is to be normalized

    Output:
        Normalized array with values between 0 and 1
    '''
    return (col-np.min(col))/(np.max(col)-np.min(col))

def scoring(df, label, week):
    '''
    This function will go through and calculate the scores for each player at their 
    given poisition. The scores are used by the algorithms cost function to figure out
    what the total "cost" would be to add a patricular player to the roster.

    Input:
        df(pd.dataframe): List of all the players for a given position
        label(str): label for the position of the players in ther
        week(int): the week of the season the daya is in.
            Week 0 is "preseaon" and uses projected numbers. The coefficients
            of the equations have to be addujsted accordingly
    
    Output:
        a dataframe with the calculated scores appended on the end.
    '''

    data = df.copy()

    # a Multiplier for the coefficients (There are 38 games in the average EPL season)
    if week == 0:
        ceof_multi = 38
    else:
        ceof_multi = 1

    # Equation to calculate the goalkeepers scores
    if label == 'GKP':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Scores'] = 10*ceof_multi - \
                          1.5*ceof_multi*data['Goals_conceded'] + \
                          2.0*ceof_multi*data['Penalties_saved'] + \
                          ceof_multi*data['Saves'] - \
                          3.0*ceof_multi*data['Red_cards'] + \
                          2.5*ceof_multi*data['Norm_Influence']

        # If the score is equal to the intercept, then the player has no stats to be considered
        # Therefore they will not be on a roster.
        
        return data[data['Scores'] != 10*ceof_multi]

    # Equation to calculate the defenders scores
    elif label == 'DEF':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Scores'] = 10*ceof_multi - \
                            ceof_multi*data['Goals_conceded'] + \
                          5*ceof_multi*data['Goals_scored'] + \
                          2.5*ceof_multi*data['Norm_Influence'] - \
                          1.5*ceof_multi*data['Norm_Threat'] + \
                          1.5*ceof_multi*data['Norm_Creativity'] - \
                          2.0*ceof_multi*data['Red_cards'] + \
                          ceof_multi*data['Assists'] - \
                          2.0*ceof_multi*data['Own_goals']  
        return data[data['Scores'] != 10*ceof_multi]     

    # Equation to calculate the Midfielders scores
    elif label == 'MID':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Scores'] = 10*ceof_multi - \
                        ceof_multi*data['Goals_conceded'] + \
                        2.0*ceof_multi*data['Goals_scored'] + \
                        2.5*ceof_multi*data['Norm_Influence'] + \
                        1.5*ceof_multi*data['Norm_Threat'] + \
                        5.0*ceof_multi*data['Norm_Creativity'] - \
                        2.0*ceof_multi*data['Red_cards'] + \
                        2.0*ceof_multi*data['Assists'] - \
                        2.0*ceof_multi*data['Own_goals']

        return data[data['Scores'] != 10*ceof_multi] 

    # Equation to calculate the Forwards scores
    else:
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Scores'] = 10*ceof_multi + \
                            2.0*ceof_multi*data['Goals_scored'] + \
                            2.5*ceof_multi*data['Norm_Influence'] + \
                            1.5*ceof_multi*data['Norm_Creativity'] + \
                            5.0*ceof_multi*data['Norm_Threat'] - \
                            2.5*ceof_multi*data['Red_cards'] + \
                            ceof_multi*data['Assists']
        return data[data['Scores'] != 10*ceof_multi] 

In [11]:
PATH = 'https://raw.githubusercontent.com/DanielBrooks253/Kaggle/main/Soccer_Team/Data/'
filenames = ['FPL_2018_19_Wk0.csv', # Projected Scores
             'FPL_2018_19_Wk1.csv',
             'FPL_2018_19_Wk2.csv',
             'FPL_2018_19_Wk3.csv',
             'FPL_2018_19_Wk4.csv',
             'FPL_2018_19_Wk5.csv',
             'FPL_2018_19_Wk6.csv',
             'FPL_2018_19_Wk7.csv',
             'FPL_2018_19_Wk8.csv',
             'FPL_2018_19_Wk9.csv',
             'FPL_2018_19_Wk10.csv',
             'FPL_2018_19_Wk11.csv']

person_lookup = {'DEF': [],
                 'GKP': [],
                 'MID': [],
                 'FWD': []}

# Get the individual weeks and also the overall combined dataset
weeks = [pd.read_csv(PATH + i) for i in filenames]
adjust_names = weeks.copy()

# Some of the names of the players repeat.
# This causes issues when trying to create dictionary keys later on.
# THerefore I concatenated the team name with the last name to make a unique key.
for idx, _ in enumerate(adjust_names):
    adjust_names[idx]['New_Name'] = adjust_names[idx]['Name'] + ', ' + adjust_names[idx]['Team']  

In [12]:
# Get the scores for all of the positions for all of the available weeks
list_player_score_df = [pd.concat(list(map(lambda x: scoring(adjust_names[i][adjust_names[i]['Position'] == x], x, i), ['DEF', 'FWD', 'GKP', 'MID'])), axis=0)
                           for i in range(0, 12)]

# Create a list of dictionaries for each week.
# The dictionary maps the players name to their scores
player_scores_weeks = [dict(zip(i['New_Name'], i['Scores'])) for i in list_player_score_df]

# Filter the weeks to only players who played 
player_score_keys = [list(i.keys()) for i in player_scores_weeks]
filtered_weeks = [i[i['New_Name'].isin(j)] for i,j in zip(adjust_names, player_score_keys)]

In [13]:
# Get the mapping from the position to the person
filter_person = [list(map(lambda x: filtered_weeks[i][filtered_weeks[i]['Position'] == x], ['DEF', 'FWD', 'GKP', 'MID'])) for i in range(0, 12)]
person_lookup = [{j['Position'].unique()[0]:np.array(j['New_Name']) for j in i} for i in filter_person]

# Different Formations (# Defenders - # Midfielders - # Forwards)

## 4-3-3

In [14]:
formation = (4,3,3)
max_gens = 100
pop_size = 30
p_crossover = .99
p_mutation = 1.0/64
week = 0

lineups = search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week)

print('\n ***************************************************************************** \n' 
'                        Best Lineup for week {0}                                    \n' 
'                         Formation: {1}-{2}-{3}                                     \n' 
'***************************************************************************** \n'                    
'GKP: {4} \n'
'DEF: {5} \n'
'MID: {6} \n'
'FWD: {7}'.format(week, formation[0], formation[1], formation[2], lineups['lineup'][0], 
                                                                  lineups['lineup'][1:(formation[0]+1)],
                                                                  lineups['lineup'][(formation[0]+1):(formation[0]+formation[1]+1)],
                                                                  lineups['lineup'][(formation[0]+formation[1]+1):]))

Gen: 0, Fitness: 12962.385622127345, Best: ['Ryan, BHA', 'Bruno, BHA', 'Hadergjonaj, HUD', 'Mavropanos, ARS', 'Williams, EVE', 'Ince, HUD', 'Kante, CHE', 'Salah, LIV', 'Vardy, LEI', 'Quaner, HUD', 'Kane, TOT'] 
Gen: 1, Fitness: 12962.385622127345, Best: ['Ryan, BHA', 'Bruno, BHA', 'Hadergjonaj, HUD', 'Mavropanos, ARS', 'Williams, EVE', 'Ince, HUD', 'Kante, CHE', 'Salah, LIV', 'Vardy, LEI', 'Quaner, HUD', 'Kane, TOT'] 
Gen: 2, Fitness: 12962.385622127345, Best: ['Ryan, BHA', 'Bruno, BHA', 'Hadergjonaj, HUD', 'Mavropanos, ARS', 'Williams, EVE', 'Ince, HUD', 'Kante, CHE', 'Salah, LIV', 'Vardy, LEI', 'Quaner, HUD', 'Kane, TOT'] 
Gen: 3, Fitness: 12962.385622127345, Best: ['Ryan, BHA', 'Bruno, BHA', 'Hadergjonaj, HUD', 'Mavropanos, ARS', 'Williams, EVE', 'Ince, HUD', 'Kante, CHE', 'Salah, LIV', 'Vardy, LEI', 'Quaner, HUD', 'Kane, TOT'] 
Gen: 4, Fitness: 13768.572117678661, Best: ['Ryan, BHA', 'Lindelof, MUN', 'Young, MUN', 'Moses, CHE', 'Darmian, MUN', 'Iwobi, ARS', 'Ward-Prowse, SOU', 'Dia

## 3-6-1

In [18]:
formation = (3,6,1)
max_gens = 100
pop_size = 30
p_crossover = .99
p_mutation = 1.0/64
week = 0

lineups = search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week)

print('\n ***************************************************************************** \n' 
'                        Best Lineup for week {0}                                    \n' 
'                         Formation: {1}-{2}-{3}                                     \n' 
'***************************************************************************** \n'                    
'GKP: {4} \n'
'DEF: {5} \n'
'MID: {6} \n'
'FWD: {7}'.format(week, formation[0], formation[1], formation[2], lineups['lineup'][0], 
                                                                  lineups['lineup'][1:(formation[0]+1)],
                                                                  lineups['lineup'][(formation[0]+1):(formation[0]+formation[1]+1)],
                                                                  lineups['lineup'][(formation[0]+formation[1]+1):]))

Gen: 0, Fitness: 9088.270823148285, Best: ['De Gea, MUN', 'Morgan, LEI', 'Alonso, CHE', 'Kenny, EVE', 'Shaqiri, LIV', 'Martial, MUN', 'Choudhury, LEI', 'Walters, BUR', 'Mata, MUN', 'Lingard, MUN', 'Gayle, NEW'] 
Gen: 1, Fitness: 11231.93001209647, Best: ['De Gea, MUN', 'Danilo, MCI', 'Alderweireld, TOT', 'Darmian, MUN', 'Sanchez, MUN', 'Gudmundsson, BUR', 'Doucoure, WAT', 'Bernardo Silva, MCI', 'Loftus-Cheek, CHE', 'Herrera, MUN', 'Lukaku, MUN'] 
Gen: 2, Fitness: 11235.546919136195, Best: ['Schmeichel, LEI', 'Danilo, MCI', 'Alderweireld, TOT', 'Darmian, MUN', 'Izquierdo, BHA', 'Henderson, LIV', 'Loftus-Cheek, CHE', 'Son, TOT', 'Sanchez, MUN', 'Bernardo Silva, MCI', 'Lukaku, MUN'] 
Gen: 3, Fitness: 12195.71140385177, Best: ['Schmeichel, LEI', 'Danilo, MCI', 'Alderweireld, TOT', 'Darmian, MUN', 'Henderson, LIV', 'Izquierdo, BHA', 'Silva, LEI', 'Hudson-Odoi, CHE', 'Alli, TOT', 'Salah, LIV', 'Lukaku, MUN'] 
Gen: 4, Fitness: 13358.560260938291, Best: ['De Gea, MUN', 'David Luiz, CHE', 'Bail

## 4-5-1

In [16]:
formation = (4,5,1)
max_gens = 100
pop_size = 30
p_crossover = .99
p_mutation = 1.0/64
week = 0

lineups = search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week)

print('\n ***************************************************************************** \n' 
'                        Best Lineup for week {0}                                    \n' 
'                         Formation: {1}-{2}-{3}                                     \n' 
'***************************************************************************** \n'                    
'GKP: {4} \n'
'DEF: {5} \n'
'MID: {6} \n'
'FWD: {7}'.format(week, formation[0], formation[1], formation[2], lineups['lineup'][0], 
                                                                  lineups['lineup'][1:(formation[0]+1)],
                                                                  lineups['lineup'][(formation[0]+1):(formation[0]+formation[1]+1)],
                                                                  lineups['lineup'][(formation[0]+formation[1]+1):]))

Gen: 0, Fitness: 10246.779816355514, Best: ['Caballero, CHE', 'Alonso, CHE', 'Laporte, MCI', 'Moreno, LIV', 'Mings, BOU', 'Mane, LIV', 'Sissoko, TOT', 'Eriksen, TOT', 'Fabregas, CHE', 'Sane, MCI', 'Wood, BUR'] 
Gen: 1, Fitness: 10544.194380920848, Best: ['Ryan, BHA', 'Riedewald, CRY', 'Kolasinac, ARS', 'Ward, BUR', 'Alonso, CHE', 'Mane, LIV', 'Dier, TOT', 'Sissoko, TOT', 'Eriksen, TOT', 'Sane, MCI', 'Welbeck, ARS'] 
Gen: 2, Fitness: 11293.001216038563, Best: ['Pickford, EVE', 'Aurier, TOT', 'Baines, EVE', 'Maguire, LEI', 'Moses, CHE', 'Fabregas, CHE', 'Sissoko, TOT', 'Sane, MCI', 'Eriksen, TOT', 'Mata, MUN', 'Lacazette, ARS'] 
Gen: 3, Fitness: 12675.652555355966, Best: ['Ryan, BHA', 'Alonso, CHE', 'Laporte, MCI', 'Emerson, CHE', 'Lovren, LIV', 'Eriksen, TOT', 'Sissoko, TOT', 'Fabregas, CHE', 'Sane, MCI', 'Deulofeu, WAT', 'Lacazette, ARS'] 
Gen: 4, Fitness: 13467.734586877086, Best: ['Ryan, BHA', 'Alonso, CHE', 'Kolasinac, ARS', 'Laporte, MCI', 'Emerson, CHE', 'Mane, LIV', 'Sane, MCI', 

## 4-4-2

In [17]:
formation = (4,4,2)
max_gens = 100
pop_size = 30
p_crossover = .99
p_mutation = 1.0/64
week = 0

lineups = search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week)

print('\n ***************************************************************************** \n' 
'                        Best Lineup for week {0}                                    \n' 
'                         Formation: {1}-{2}-{3}                                     \n' 
'***************************************************************************** \n'                    
'GKP: {4} \n'
'DEF: {5} \n'
'MID: {6} \n'
'FWD: {7}'.format(week, formation[0], formation[1], formation[2], lineups['lineup'][0], 
                                                                  lineups['lineup'][1:(formation[0]+1)],
                                                                  lineups['lineup'][(formation[0]+1):(formation[0]+formation[1]+1)],
                                                                  lineups['lineup'][(formation[0]+formation[1]+1):]))

Gen: 0, Fitness: 10120.001631088828, Best: ['Hennessey, CRY', 'Alonso, CHE', 'Christensen, CHE', 'Bruno, BHA', 'Matip, LIV', 'Hudson-Odoi, CHE', 'Salah, LIV', 'Son, TOT', 'Lanzini, WHU', 'Austin, SOU', 'Sorloth, CRY'] 
Gen: 1, Fitness: 11621.670502103168, Best: ['Fabianski, WHU', 'Rudiger, CHE', 'Martina, EVE', 'Moses, CHE', 'Robertson, LIV', 'Pogba, MUN', 'Loftus-Cheek, CHE', 'Lallana, LIV', 'Mane, LIV', 'Vokes, BUR', 'Vardy, LEI'] 
Gen: 2, Fitness: 12308.404719041728, Best: ['Fabianski, WHU', 'Rudiger, CHE', 'Janmaat, WAT', 'Clyne, LIV', 'Souare, CRY', 'Pogba, MUN', 'Lallana, LIV', 'Winks, TOT', 'Alli, TOT', 'Vardy, LEI', 'Okazaki, LEI'] 
Gen: 3, Fitness: 12367.846900732722, Best: ['Fabianski, WHU', 'Janmaat, WAT', 'Rudiger, CHE', 'Martina, EVE', 'Moses, CHE', 'Pogba, MUN', 'Lallana, LIV', 'Winks, TOT', 'Alli, TOT', 'Vardy, LEI', 'Perez, NEW'] 
Gen: 4, Fitness: 12974.921878613333, Best: ['Pope, BUR', 'Rudiger, CHE', 'Janmaat, WAT', 'Clyne, LIV', 'Souare, CRY', 'Mane, LIV', 'Iborra, L

# Best Team for Lowest Player Cost